<a href="https://colab.research.google.com/github/mariasakorafou/maria1/blob/main/Askisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
#εγκατάσταση πακέτου PyOD (https://pyod.readthedocs.io/en/latest/)
!pip install pyod
!pip install ppscore matplotlib seaborn pandas
!pip install xgboost

In [14]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier





In [15]:
# Load the dataset
# Ensure you have the correct file path or URL
file_path = "https://raw.githubusercontent.com/kdemertzis/EKPA/refs/heads/main/Data/DarkNet.csv"
dataset = pd.read_csv(file_path)


<ipython-input-15-096b7124ec04>:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset = pd.read_csv(file_path)


In [16]:
# Data Cleaning
# Drop unnecessary columns
dataset = dataset.drop(columns=['Unnamed: 1', 'Label-1.1', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 6'])


KeyError: "['Unnamed: 1', 'Label-1.1', 'Unnamed: 3', 'Unnamed: 5', 'Unnamed: 6'] not found in axis"

In [4]:


# Normalize 'Label-2'
dataset['Label-2'] = dataset['Label-2'].str.upper().str.replace('-', '_')

# Convert 'Label-1' to binary (Tor: 1, Non-Tor: 0)
dataset['Label-1'] = dataset['Label-1'].apply(lambda x: 1 if x.upper() == 'TOR' else 0)



In [5]:
# Oversampling the minority class (Tor)
tor_samples = dataset[dataset['Label-1'] == 1]
non_tor_samples = dataset[dataset['Label-1'] == 0]




In [6]:
# Manually oversample Tor to balance the dataset
oversampled_tor = tor_samples.sample(n=len(non_tor_samples), replace=True, random_state=42)
balanced_dataset = pd.concat([non_tor_samples, oversampled_tor]).sample(frac=1, random_state=42).reset_index(drop=True)


In [7]:

# Prepare features and labels
X = balanced_dataset['Label-2'].str.get_dummies()
y = balanced_dataset['Label-1']



In [8]:
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [9]:
# Random Forest Classifier
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [10]:

# Evaluate Random Forest
rf_predictions = rf_model.predict(X_test)
rf_report = classification_report(y_test, rf_predictions)
rf_conf_matrix = confusion_matrix(y_test, rf_predictions)
print("Random Forest Classification Report:\n", rf_report)
print("Random Forest Confusion Matrix:\n", rf_conf_matrix)


Random Forest Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.62      0.65     20157
           1       0.65      0.71      0.68     20156

    accuracy                           0.67     40313
   macro avg       0.67      0.67      0.66     40313
weighted avg       0.67      0.67      0.66     40313

Random Forest Confusion Matrix:
 [[12569  7588]
 [ 5902 14254]]


In [12]:
# XGBoost Classifier
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [20:37:05] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The XGBClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)